In [39]:
import boto3
import numpy as np
from botocore.exceptions import ClientError

import requests
import pdfplumber
from markdownify import markdownify as md

In [40]:
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Set the model ID, e.g., Titan Text Premier.
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

In [41]:
def query(client, message):
    conversation = [
        {
            "role": "user",
            "content": [{"text": message}],
        }
    ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=conversation,
            inferenceConfig={"maxTokens": 4096,"temperature": 1},
            additionalModelRequestFields={"top_k":250}
        )

        # Extract and print the response text.
        response_text = response["output"]["message"]["content"][0]["text"]
        return response_text

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)


In [42]:
# Extract text from PDF
def pdf_to_markdown(pdf_path):
    simple_text = ""
    markdown_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                simple_text += text
                markdown_text += md(text)  # Convert to Markdown format
    return markdown_text, simple_text

markdown_content, content = pdf_to_markdown("report.pdf")

In [56]:
idx = markdown_content.find("Discussion and Analysis")
print(markdown_content[idx-200:idx + 15000])

tures
40 Review of Consolidated Performance
43 Quarterly Results
46 Overview of Financial Position
13 \| ROGERS COMMUNICATIONS INC. 2023 ANNUAL REPORTMANAGEMENT’S
DISCUSSION
AND
ANALYSIS
Management’s Discussion and Analysis
This Management’s Discussion and Analysis (MD\&A) contains have been reasonable at the time they were applied but may
important information about our business and our performance for prove to be incorrect; and
the year ended December 31, 2023\. This MD\&A should be read in • was approved by our management on the date of this MD\&A.
conjunction with our 2023 Audited Consolidated Financial
Our forward\-looking information includes forecasts and projections
Statements, which have been prepared in accordance with
related to the following items, among others:
International Financial Reporting Standards (IFRS) as issued by the
• revenue;
International Accounting Standards Board (IASB).
• total service revenue;
All dollar amounts are in Canadian dollars unless otherwise st

In [58]:
message = f"""
Based on this financial report:

{markdown_content[idx-2800:idx + 15000]}

what are key financial information highlights of 2023, and compare it with 2022?
provide your answer in a presentable json object
"""
response = query(client, message)
print(response)

Here are some key financial information highlights of 2023 compared to 2022, presented in a JSON object:

{
  "Financial Highlights": [
    {
      "Metric": "Total Revenue",
      "2023": 19308,
      "2022": 15396,
      "Change": "25% increase"
    },
    {
      "Metric": "Total Service Revenue",
      "2023": 16845,
      "2022": 13305,
      "Change": "27% increase"
    },
    {
      "Metric": "Adjusted EBITDA",
      "2023": 8581,
      "2022": 6393,
      "Change": "34% increase"
    },
    {
      "Metric": "Adjusted EBITDA Margin",
      "2023": 44.4,
      "2022": 41.5,
      "Change": "2.9 percentage point increase"
    },
    {
      "Metric": "Net Income",
      "2023": 849,
      "2022": 1680,
      "Change": "49% decrease"
    },
    {
      "Metric": "Adjusted Net Income",
      "2023": 2406,
      "2022": 1915,
      "Change": "26% increase"
    },
    {
      "Metric": "Capital Expenditures",
      "2023": 3934,
      "2022": 3075,
      "Change": "28% increase"
   

In [26]:
user_message = f"""Based on the following report, retrieve the following information:
1 - what are the number of stockholders of class A stock?
2 - what are the number of stockholders of class B stock?
3 - what are the number of stockholders of class C stock?

4 - what is the composition of class A, B and C shares bought in the last quarter of 2023?

<report>
{text[120000:132000]}
</report>

"""
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]
try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens":4096,"temperature":1},
        additionalModelRequestFields={"top_k":250}
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

Based on the report, here are the answers to your questions:

1 - The report states there were approximately 7,305 stockholders of record of Class A stock as of December 31, 2023.

2 - The report states there were approximately 59 stockholders of record of Class B stock as of December 31, 2023. 

3 - The report states there were approximately 1,757 stockholders of record of Class C stock as of December 31, 2023.

4 - The report does not provide information on the specific composition of Class A, B and C shares bought in the last quarter of 2023. It only provides the total number of shares repurchased during that quarter across all three classes - 26,622,000 Class A shares and 91,645,000 Class C shares.


In [60]:
from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_path = "report.pdf"
loader = PyPDFLoader(pdf_path)

# Load and split the PDF into chunks
documents = loader.load_and_split()



In [68]:
documents

[Document(metadata={'source': 'report.pdf', 'page': 0}, page_content='A record-\nbreaking year. \n2023 Annual report'),
 Document(metadata={'source': 'report.pdf', 'page': 1}, page_content='ROGERS COMMUNICATIONS INC. 2023 ANNUAL REPORT1'),
 Document(metadata={'source': 'report.pdf', 'page': 2}, page_content='About  \nRogers.\nTed Rogers started our company with one small loan \nand one big dream. He believed in the power of \ncommunication to inform, inspire and innovate. Driven  \nto honour his father’s legacy, he purchased his very first \nradio station at the age of 27.  \nFrom these humble beginnings, we’ve grown into a \nworld-class media and communications company - a \ncompany that’s driven to honour our founder’s legacy \nand to bring our customers and our communities the \nvery best products, services and experiences.  \nAfter all, connectivity is at the centre of your life and \nwe want you to be connected, at home and on the go. \nWhether you’re watching the final minute of 

In [ ]:
print(documents[50].page_content)